In [1]:
import sys
sys.path.append('..')

import lcatools
import os

In [2]:
import xlrd
import re

In [3]:
from collections import defaultdict

In [4]:
lcia_file = '/data/LCI/LCIA implementation v3.1 2014_08_13.xlsx'
catalog_dir = os.path.join(os.path.expanduser('~'), 'GitHub', 'lca-tools-datafiles', 'catalogs')


In [5]:
%time X = xlrd.open_workbook(lcia_file)

CPU times: user 19.2 s, sys: 20 ms, total: 19.2 s
Wall time: 19.2 s


In [10]:
s = X.sheet_by_index(1)

In [11]:
s.name

'impact methods'

In [12]:
b = X.sheet_by_name('impact methods')

In [62]:
headings

['method',
 'category',
 'indicator',
 'name',
 'compartment',
 'subcompartment',
 'CF 3.01',
 'CF 3.1',
 'Change?',
 'note',
 'Known issue']

In [46]:
headings[0].value

'method'

In [52]:
g = b.get_rows()
headings = [h.value for h in next(g)]

In [53]:
headings[0]

'method'

In [54]:
cfs = []
for row in g:
    d = dict()
    for i,h in enumerate(headings):
        d[h] = row[i]
    cfs.append(d)

In [55]:
len(cfs)

170915

In [56]:
cfs[0]

{'CF 3.01': number:1.6,
 'CF 3.1': number:1.6,
 'Change?': text:'FALSE',
 'Known issue': empty:'',
 'category': text:'acidification potential',
 'compartment': text:'air',
 'indicator': text:'average European',
 'method': text:'CML 2001',
 'name': text:'Ammonia',
 'note': text:'',
 'subcompartment': text:'low population density, long-term'}

In [6]:
from lcatools.providers.ecoinvent_lcia import EcoinventLcia

In [7]:
lcia_file

'/data/LCI/LCIA implementation v3.1 2014_08_13.xlsx'

In [8]:
EL = EcoinventLcia(lcia_file)

Adding quantity entity with 75c16700-535a-469a-a4d2-6f7831271a57: Mass


In [9]:
EL._create_all_quantities()

Adding quantity entity with 5eb0ce92-9c4d-3743-aade-35f58a19eeb1: CML 2001, acidification potential, average European
Adding quantity entity with 663620fb-ea1a-3cd6-a242-521e65a77a5d: CML 2001, acidification potential, generic
Adding quantity entity with b5fe2cea-0418-3a38-9093-5f936b2d20d5: CML 2001, climate change, GWP 100a
Adding quantity entity with dcf44476-1b29-30f7-bdbc-02ad2bc38f74: CML 2001, climate change, GWP 20a
Adding quantity entity with c00fb9b8-aa83-3c9f-a808-656266ef7c45: CML 2001, climate change, GWP 500a
Adding quantity entity with 726bc194-4b18-3afc-892c-b74ce9ce3b2a: CML 2001, climate change, lower limit of net GWP
Adding quantity entity with 4f61bf8e-9795-36db-b034-2409c2b58b27: CML 2001, climate change, upper limit of net GWP
Adding quantity entity with 66c2ff7a-bdf1-3654-91db-34927fcb09c6: CML 2001, eutrophication potential, average European
Adding quantity entity with e6c6bb8f-5fcc-30b0-8130-21c3970a873e: CML 2001, eutrophication potential, generic
Adding quant

In [10]:
%time EL._load_xl_rows()

CPU times: user 22.8 s, sys: 100 ms, total: 22.9 s
Wall time: 22.9 s


In [11]:
len(EL._xl_rows)

170915

In [12]:
for row in EL._xl_rows[:10]:
    EL._create_flow(row)

Adding flow entity with 0b5402d3-df93-399f-8e88-f032ff10f325: Ammonia
Adding flow entity with bdd7a9d6-1557-3a2e-94ed-dd3b09a14546: Ammonia
Adding flow entity with 88b39939-7951-3d33-ac2a-88f97f8391f1: Ammonia
Adding flow entity with 8ed7fbb1-22ae-3a55-8567-d22f06b02738: Ammonia
Adding flow entity with e80cea4c-dc01-3365-a63a-7700cde4402d: Ammonia
Adding flow entity with b59c4154-6d6e-3175-a0e6-f0aa9ebf9ecf: Nitrogen oxides
Adding flow entity with 7899a56c-1ac3-3871-90c0-144216094601: Nitrogen oxides
Adding flow entity with 2af5f4c4-c17a-3242-b0e2-1897b8dfd6f3: Nitrogen oxides
Adding flow entity with d1eb5fc2-8669-3a8f-9c4a-309136398bd2: Nitrogen oxides
Adding flow entity with 27ea9823-d24f-3774-9874-edd9ac18769f: Nitrogen oxides


In [13]:
EL.flows()[0].get_signature()

{'CasNumber': '',
 'Comment': '',
 'Compartment': ['air', 'low population density, long-term'],
 'EntityType': 'flow',
 'Name': 'Ammonia',
 'referenceQuantity': <lcatools.entities.LcQuantity at 0x7fb483ee9828>}

In [14]:
EL._quiet=True

In [15]:
%time EL.load_all()

0 new process entities added (0 total)
3255 new flow entities added (3255 total)
702 new quantity entities added (702 total)
CPU times: user 10.6 s, sys: 50 ms, total: 10.6 s
Wall time: 10.6 s


In [16]:
len(EL.characterizations())

170755

In [17]:
EL.characterizations()[0].quantity.get_uuid()

'e709b2f3-3ce8-3ed4-8520-d9840eb6e134'

In [52]:
def flows_gen():
    for _f in EL.flows():
        yield _f

In [53]:
flows = flows_gen()

In [69]:
%timeit -n3 print(len(EL._characterizations.get(flow=next(flows))))

68
177
38
23
11
5
68
63
4
3 loops, best of 3: 56.3 ms per loop


In [54]:
f = next(flows)

In [55]:
f

In [66]:
len(EL._characterizations.get(flow=next(flows)))

1

In [61]:
f=next(flows)
f.get_signature()

{'CasNumber': '',
 'Comment': '',
 'Compartment': ['air', 'non-urban air or from high stacks'],
 'EntityType': 'flow',
 'Name': 'Dipropylamine',
 'referenceQuantity': <lcatools.entities.LcQuantity at 0x7fb483ee9828>}

In [51]:
len([f for f in flows()])

3255

In [71]:
cf = EL._characterizations.get(flow=next(flows))

In [72]:
len(cf)

152

In [75]:
cf[0].__dict__

{'flow': <lcatools.entities.LcFlow at 0x7fb44b5dd710>,
 'quantity': <lcatools.entities.LcQuantity at 0x7fb483ed0470>,
 'value': 224400.0}

In [79]:
def cf_str(cf):
    return { 'flow': str(cf.flow), 'quantity': str(cf.quantity), 'value': cf.value}

In [86]:
cf_str(cf[2])

{'flow': 'Cadmium, ion [water, ground-]',
 'quantity': 'LC quantity: ReCiPe Endpoint (H,A) w/o LT, ecosystem quality w/o LT, freshwater ecotoxicity w/o LT',
 'value': 0.0172175497759}

In [90]:
cf[2].value.keys()

dict_keys(['category', 'indicator', 'method', 'name', 'note', 'CF 3.1', 'CF 3.01', 'subcompartment', 'Known issue', 'compartment'])

In [103]:
cf[3].value['indicator']

'carcinogenics'

In [104]:
unique_names = set([f['Name'] for f in EL.flows()])

In [105]:
len(unique_names)

984

In [107]:
list(unique_names)[:10]

['Occupation, permanent crop, irrigated',
 'Magnesite, 60% in crude ore, in ground',
 'Chromium, 25.5% in chromite, 11.6% in crude ore, in ground',
 'Molybdenum, 0.022% in sulfide, Mo 8.2E-3% and Cu 0.22% in crude ore, in ground',
 'Lead-210',
 'Copper, Cu 0.38%, in mixed ore, in ground',
 'Prosulfuron',
 'Transformation, to heterogeneous, agricultural',
 'Ethane, 1,1,1,2-tetrafluoro-, HFC-134a',
 'Transformation, from seabed, unspecified']

In [32]:
char_dict = dict()
for i in EL.characterizations():
    char_dict['_'.join([i.flow.get_uuid(), i.quantity.get_uuid()])] = i

In [51]:
f = EL.flows()[123]
q = EL.quantities()[698]

In [56]:
%timeit -n5 [i for i in EL.characterizations() if i.flow is f and i.quantity is q]

5 loops, best of 3: 77.9 ms per loop


In [55]:
%timeit -n5 char_dict['_'.join([f.get_uuid(), q.get_uuid()])]

5 loops, best of 3: 8.03 µs per loop


In [49]:
q

In [50]:
[i for i, v in enumerate(EL.quantities()) if v is q]

[698]

In [58]:
b = [1, 2, 3, 0, 4]

In [66]:
def _to_dict(i):
    if i == 0:
        return None
    return {'value': i}

g =  filter(None, [_to_dict(i) for i in b])

TypeError: 'filter' object is not callable

In [17]:
EL.check_counter()

0 new process entities added (0 total)
0 new flow entities added (0 total)
702 new quantity entities added (702 total)


In [24]:
EL.quantities()[0]['referenceUnit'].unitstring()

'points'

In [18]:
EL._xl_rows[0]

{'CF 3.01': 1.6,
 'CF 3.1': 1.6,
 'Change?': 'FALSE',
 'Known issue': '',
 'category': 'acidification potential',
 'compartment': 'air',
 'indicator': 'average European',
 'method': 'CML 2001',
 'name': 'Ammonia',
 'note': '',
 'subcompartment': 'low population density, long-term'}

In [17]:
f = EL.flows()[0]

In [22]:
EL.load_all()

0 new process entities added (0 total)
3245 new flow entities added (3255 total)
0 new quantity entities added (702 total)


In [32]:
cf = EL.characterizations()[100000]

In [33]:
cf.__dict__

{'flow': <lcatools.entities.LcFlow at 0x7fa002887f60>,
 'quantity': <lcatools.entities.LcQuantity at 0x7fa009760208>,
 'value': None}

In [38]:
EL.write_to_file(os.path.join(catalog_dir, 'ei_lcia.gz'), gzip=True, characterizations=False)

In [28]:
EL.flows()[100].get_signature()

{'CasNumber': '',
 'Comment': '',
 'Compartment': ['air', 'urban air close to ground'],
 'EntityType': 'flow',
 'Name': 'Cesium-137',
 'referenceQuantity': <lcatools.entities.LcQuantity at 0x7fa009c45748>}